In [19]:
!pip install virtualenv
!virtualenv venv

created virtual environment CPython3.10.12.final.0-64 in 1062ms
  creator CPython3Posix(dest=/content/venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.2.1, setuptools==68.0.0, wheel==0.41.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [20]:
!source venv/bin/activate

In [21]:
# Define custom intents
custom_intents = ["animal_info", "habitat_info"]


In [22]:
# Example dataset
dataset = [
    {"text": "Tell me about lions.", "intent": "animal_info"},
    {"text": "Where do tigers live?", "intent": "habitat_info"},
    # ... more examples ...
]


In [23]:
!pip install transformers

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")  # Choose any suitable model

# Tokenize and format data
inputs = tokenizer([example["text"] for example in dataset], padding=True, truncation=True, return_tensors="pt")
labels = [custom_intents.index(example["intent"]) for example in dataset]


In [15]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

model_name = "distilbert-base-uncased"
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(custom_intents))
tokenizer = DistilBertTokenizer.from_pretrained(model_name)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
!pip install torch

In [17]:
# Assuming you have prepared the dataset as shown before

# Tokenize and format data
encoded_dataset = tokenizer([example["text"] for example in dataset], padding=True, truncation=True, return_tensors="pt")

# Fine-tuning setup
from torch.utils.data import TensorDataset

input_ids = encoded_dataset["input_ids"]
attention_mask = encoded_dataset["attention_mask"]
labels = torch.tensor(labels)

dataset = TensorDataset(input_ids, attention_mask, labels)


NameError: ignored

In [ ]:
# !pip install transformers[torch]
!pip install transformers==4.8.2 accelerate==0.5.1



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 29.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895240 sha256=4a37ce907eb5509f768c80287d80ec0c5e88304fb6584691bba6a983b27bd47a
  Stored in directory: /root/.cache/pip/whee

In [ ]:
!pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
!pip install --upgrade setuptools wheel


  Obtaining dependency information for setuptools from https://files.pythonhosted.org/packages/b8/49/b3b29c52b09075fb77f69309763a563b4054d5808a3f3b95df3a62ef3d3f/setuptools-68.1.0-py3-none-any.whl.metadata
Using cached setuptools-68.1.0-py3-none-any.whl (805 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
sudo apt-get update
sudo apt-get install build-essential


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./intent_classification_results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()


ImportError: ignored

In [ ]:
def get_intent(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    intent_idx = torch.argmax(outputs.logits).item()
    return custom_intents[intent_idx]


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

class IntentDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.inputs.items()}, self.labels[idx]

# Create DataLoader
intent_dataset = IntentDataset(inputs, labels)
intent_dataloader = DataLoader(intent_dataset, batch_size=16, shuffle=True)

# Define a simple classifier model
class IntentClassifier(nn.Module):
    def __init__(self, num_intents):
        super(IntentClassifier, self).__init__()
        self.embedding = nn.Embedding(tokenizer.vocab_size, 128)  # You might need to adjust the embedding size
        self.classifier = nn.Linear(128, num_intents)  # Adjust the input size to match the embedding size

    def forward(self, inputs):
        embedded = self.embedding(inputs["input_ids"])
        flattened_embedded = embedded.view(embedded.size(0), -1)
        logits = self.classifier(flattened_embedded)
        return logits


# Initialize model and optimizer
num_intents = len(custom_intents)
model = IntentClassifier(num_intents)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for batch in intent_dataloader:
        optimizer.zero_grad()
        inputs, labels = batch
        logits = model(inputs)
        loss = nn.CrossEntropyLoss()(logits, labels)
        loss.backward()
        optimizer.step()


RuntimeError: ignored